In [1]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import random
import time

from bokeh.io import show, output_notebook
from bokeh.plotting import figure, gridplot
from bokeh.models import LinearAxis, Range1d
output_notebook()

Loading BokehJS ...

### GPU
指定使用的 GPU 编号。  
`watch -n 1 nvidia-smi` 实时查看 GPU 的运行状态。 

### Data
读入 dataset 中的 stock.xlsx，将开盘、最高、最低、收盘价格作为特征，涨跌作为标签（比前一天收盘价高即为涨）。

In [2]:
df = pd.read_excel('../dataset/stock.xlsx')
df

,名称,代码,日期,开盘,最高,最低,收盘,涨跌
0,上证指数,980001,2017-01-03,3105.31,3136.46,3105.31,3135.92,1
1,上证指数,980001,2017-01-04,3133.79,3160.10,3130.11,3158.79,1
2,上证指数,980001,2017-01-05,3157.91,3168.50,3154.28,3165.41,1
3,上证指数,980001,2017-01-06,3163.78,3172.03,3153.03,3154.32,0
4,上证指数,980001,2017-01-09,3148.53,3173.14,3147.74,3171.24,1
...,...,...,...,...,...,...,...,...
177,上证指数,980001,2017-09-21,3364.70,3377.88,3356.88,3357.81,0
178,上证指数,980001,2017-09-22,3347.16,3356.45,3334.98,3352.53,0
179,上证指数,980001,2017-09-25,3344.59,3350.96,3334.94,3341.55,0
180,上证指数,980001,2017-09-26,3336.35,3347.16,3332.60,3343.58,1


In [3]:
mean_value = df.iloc[:, 6].mean()

In [4]:
x = torch.FloatTensor(df.iloc[:100, 3:7].values) - mean_value
y = torch.FloatTensor(df.iloc[1:101, 7].values).unsqueeze(1)

### 线性模型

In [7]:
class Net(nn.Module):
    def __init__(self, dim_in, dim_hid, dim_out):
        super().__init__()
        self.hidden = nn.Linear(dim_in, dim_hid)
        self.predict = nn.Linear(dim_hid, dim_out)
    def forward(self, x):
        x = torch.relu(self.hidden(x))
        x = self.predict(x)
        return torch.tanh(x)

In [8]:
lrate = 0.001
epochs = 8000

model = Net(4, 10, 1)
criterion = nn.MSELoss()
optim = torch.optim.SGD(model.parameters(), lr=lrate)

In [9]:
result = []

for e in range(epochs):
    pred = model(x)
    loss = criterion(pred, y)
    optim.zero_grad()
    loss.backward()
    optim.step()
    if e % 100 == 0:
        result.append(float(loss))
fig = figure()
fig.line(range(len(result)), result)
show(fig)

In [10]:
model.eval()
pred = model(x)
ans = torch.round(pred) - y
acc = 1-ans.squeeze().abs().sum()/ans.shape[0]
acc

tensor(0.6800, grad_fn=<RsubBackward1>)

### Save Model